In [1]:
! pip install pandas openpyxl

In [4]:
#@title создание json с целочисленными азимутами и именем камеры и исключениями по имени и по типу
import pandas as pd
import os
import json
from google.colab import drive  # Удалите или закомментируйте, если не в Colab

# === НАСТРОЙКИ ПУТЕЙ ===
# Если в Google Colab — раскомментируйте следующую строку и смонтируйте диск:
# drive.mount('/content/drive')

xlsx_path = '/content/drive/MyDrive/useful/generate_json/v.xlsx'
output_file = '/content/drive/MyDrive/useful/v_json/v_name.json'

# === ПОДГОТОВКА ДИРЕКТОРИИ ===
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Удаление старого JSON, если существует
if os.path.exists(output_file):
    os.remove(output_file)
    print(f"🗑️ Удалён старый файл: {output_file}")

# === ЧТЕНИЕ EXCEL ===
try:
    df = pd.read_excel(xlsx_path)
except FileNotFoundError:
    raise FileNotFoundError(f"❌ Файл не найден: {xlsx_path}")
except Exception as e:
    raise Exception(f"❌ Ошибка при чтении Excel: {e}")

# === ПРОВЕРКА СТОЛБЦОВ ===
required_columns = ['name', 'lat', 'lng', 'district', 'address', 'cameraType', 'defaultAzimuth']
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    raise ValueError(f"❌ В Excel отсутствуют столбцы: {missing_cols}")

df = df[required_columns].copy()

# === 1. ОЧИСТКА КООРДИНАТ ===
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lng'] = pd.to_numeric(df['lng'], errors='coerce')
initial_count = len(df)
df = df.dropna(subset=['lat', 'lng']).copy()
print(f"📍 Координаты обработаны: удалено {initial_count - len(df)} строк с некорректными lat/lng")

# === 2. УДАЛЕНИЕ СТРОК С ПУСТЫМ name ===
df = df[df['name'].notna()].copy()
df['name'] = df['name'].astype(str)

# === 3. ФИЛЬТРАЦИЯ ПО ЗАПРЕЩЁННЫМ ПРЕФИКСАМ В name ===
forbidden_prefixes = ['MDZ', 'GR',  'КПА', 'ФВФ', 'ПРОВ', 'МС', 'ЖФ', 'У',  'test', 'М', 'ТЕСТ', 'САП',  'MAZ']
pattern = r'(?i)^' + '|'.join(forbidden_prefixes)  # (?i) — в начале!
mask = df['name'].astype(str).str.contains(pattern, na=False, regex=True)
filtered_df = df[~mask].copy()

excluded_count = len(df) - len(filtered_df)
print(f"🚫 Исключено по префиксам  {forbidden_prefixes}: {excluded_count} строк")

# === 4. ФИЛЬТРАЦИЯ ПО cameraType - список запрещённых значений ===
forbidden_camera_types = ['8', 'КПА']  # Можно добавить больше значений, например: ['8', '801', '802']
camera_type_mask = filtered_df['cameraType'].astype(str).isin(forbidden_camera_types)
filtered_df = filtered_df[~camera_type_mask].copy()
camera_type_excluded = len(df) - len(filtered_df) - excluded_count
print(f"🚫 Исключено по cameraType из списка запрещённых: {camera_type_excluded} строк")

# === 5. ОБРАБОТКА defaultAzimuth (после всех фильтров!) ===
filtered_df['defaultAzimuth'] = pd.to_numeric(filtered_df['defaultAzimuth'], errors='coerce')
azimuth_na_count = filtered_df['defaultAzimuth'].isna().sum()
if azimuth_na_count > 0:
    print(f"⚠️ Удалено {azimuth_na_count} строк из-за отсутствия значения defaultAzimuth")
filtered_df = filtered_df.dropna(subset=['defaultAzimuth']).copy()

# Теперь можно безопасно конвертировать в int
filtered_df['defaultAzimuth'] = filtered_df['defaultAzimuth'].round().astype(int)

# === 6. ФИНАЛЬНАЯ ОЧИСТКА (на всякий случай) ===
final_data = filtered_df.dropna(subset=['name', 'district', 'address', 'cameraType']).copy()
final_data = final_data.reset_index(drop=True)

# === 7. СОХРАНЕНИЕ В JSON ===
data = final_data.to_dict(orient='records')

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# === РЕЗУЛЬТАТ ===
print(f"✅ Успешно сохранено в: {output_file}")
print(f"📊 Общее количество записей в JSON: {len(data)}")

🗑️ Удалён старый файл: /content/drive/MyDrive/useful/v_json/v_name.json
📍 Координаты обработаны: удалено 0 строк с некорректными lat/lng
🚫 Исключено по префиксам  ['MDZ', 'GR', 'КПА', 'ФВФ', 'ПРОВ', 'МС', 'ЖФ', 'У', 'test', 'М', 'ТЕСТ', 'САП', 'MAZ']: 28831 строк
🚫 Исключено по cameraType из списка запрещённых: 930 строк
⚠️ Удалено 2782 строк из-за отсутствия значения defaultAzimuth
✅ Успешно сохранено в: /content/drive/MyDrive/useful/v_json/v_name.json
📊 Общее количество записей в JSON: 93437


In [ ]:
#@title создание json с целочисленными азимутами и именем камеры и исключениями
import pandas as pd
import os
import json
from google.colab import drive  # Удалите или закомментируйте, если не в Colab

# === НАСТРОЙКИ ПУТЕЙ ===
# Если в Google Colab — раскомментируйте следующую строку и смонтируйте диск:
# drive.mount('/content/drive')

xlsx_path = '/content/drive/MyDrive/useful/generate_json/v.xlsx'
output_file = '/content/drive/MyDrive/useful/v_json/v_name.json'

# === ПОДГОТОВКА ДИРЕКТОРИИ ===
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Удаление старого JSON, если существует
if os.path.exists(output_file):
    os.remove(output_file)
    print(f"🗑️ Удалён старый файл: {output_file}")

# === ЧТЕНИЕ EXCEL ===
try:
    df = pd.read_excel(xlsx_path)
except FileNotFoundError:
    raise FileNotFoundError(f"❌ Файл не найден: {xlsx_path}")
except Exception as e:
    raise Exception(f"❌ Ошибка при чтении Excel: {e}")

# === ПРОВЕРКА СТОЛБЦОВ ===
required_columns = ['name', 'lat', 'lng', 'district', 'address', 'cameraType', 'defaultAzimuth']
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    raise ValueError(f"❌ В Excel отсутствуют столбцы: {missing_cols}")

df = df[required_columns].copy()

# === 1. ОЧИСТКА КООРДИНАТ ===
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lng'] = pd.to_numeric(df['lng'], errors='coerce')
initial_count = len(df)
df = df.dropna(subset=['lat', 'lng']).copy()
print(f"📍 Координаты обработаны: удалено {initial_count - len(df)} строк с некорректными lat/lng")

# === 2. УДАЛЕНИЕ СТРОК С ПУСТЫМ name ===
df = df[df['name'].notna()].copy()
df['name'] = df['name'].astype(str)

# === 3. ФИЛЬТРАЦИЯ ПО ЗАПРЕЩЁННЫМ ПРЕФИКСАМ В name ===
forbidden_prefixes = ['MDZ', 'GR', 'ФВФ', 'ПРОВ', 'МС', 'ЖФ', 'У',  'test', 'М', 'ТЕСТ', 'САП',  'MAZ']
pattern = r'(?i)^' + '|'.join(forbidden_prefixes)  # (?i) — в начале!
mask = df['name'].astype(str).str.contains(pattern, na=False, regex=True)
filtered_df = df[~mask].copy()

excluded_count = len(df) - len(filtered_df)
print(f"🚫 Исключено по префиксам (MDZ, GR, ФВФ, ПРОВО, ТЕСТ, САП,  MAZ): {excluded_count} строк")

# === 4. ОБРАБОТКА defaultAzimuth (после всех фильтров!) ===
filtered_df['defaultAzimuth'] = pd.to_numeric(filtered_df['defaultAzimuth'], errors='coerce')
azimuth_na_count = filtered_df['defaultAzimuth'].isna().sum()
if azimuth_na_count > 0:
    print(f"⚠️ Удалено {azimuth_na_count} строк из-за отсутствия значения defaultAzimuth")
filtered_df = filtered_df.dropna(subset=['defaultAzimuth']).copy()

# Теперь можно безопасно конвертировать в int
filtered_df['defaultAzimuth'] = filtered_df['defaultAzimuth'].round().astype(int)

# === 5. ФИНАЛЬНАЯ ОЧИСТКА (на всякий случай) ===
final_data = filtered_df.dropna(subset=['name', 'district', 'address', 'cameraType']).copy()
final_data = final_data.reset_index(drop=True)

# === 6. СОХРАНЕНИЕ В JSON ===
data = final_data.to_dict(orient='records')

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# === РЕЗУЛЬТАТ ===
print(f"✅ Успешно сохранено в: {output_file}")
print(f"📊 Общее количество записей в JSON: {len(data)}")

🗑️ Удалён старый файл: /content/drive/MyDrive/useful/v_json/v_name.json
📍 Координаты обработаны: удалено 0 строк с некорректными lat/lng
🚫 Исключено по префиксам (MDZ, GR, ФВФ, ПРОВО, ТЕСТ, САП,  MAZ): 28831 строк
⚠️ Удалено 2791 строк из-за отсутствия значения defaultAzimuth
✅ Успешно сохранено в: /content/drive/MyDrive/useful/v_json/v_name.json
📊 Общее количество записей в JSON: 94357


In [ ]:
!rm /content/drive/MyDrive/useful/v_json/v_name.json

#@title создание json с целочисленными азимутами и именем камеры
import pandas as pd

# Чтение данных из файла XLSX
file_path = '/content/drive/MyDrive/useful/generate_json/v.xlsx'  # Путь к вашему файлу XLSX
df = pd.read_excel(file_path)

# Выбор нужных столбцов и фильтрация строк с пустыми значениями
required_columns = ['name','lat', 'lng', 'district', 'address', 'cameraType', 'defaultAzimuth']
filtered_df = df[required_columns].dropna()

# Округление defaultAzimuth до целого числа
filtered_df['defaultAzimuth'] = filtered_df['defaultAzimuth'].round().astype(int)

# Преобразование данных в список словарей
data = filtered_df.to_dict(orient='records')

# Сохранение данных в JSON-файл
output_file = '/content/drive/MyDrive/useful/v_json/v_name.json'
with open(output_file, 'w', encoding='utf-8') as f:
    pd.DataFrame(data).to_json(f, orient='records', force_ascii=False, indent=4)

print(f"JSON-файл {output_file} успешно создан.")

JSON-файл /content/drive/MyDrive/useful/v_json/v_name.json успешно создан.


In [ ]:
#@title создание json с исходными азимутами
import pandas as pd

# Чтение данных из файла XLSX
file_path = '/content/drive/MyDrive/Вспомогательные_скрипты/jinja/v.xlsx'  # Путь к вашему файлу XLSX
df = pd.read_excel(file_path)

# Выбор нужных столбцов и фильтрация строк с пустыми значениями
required_columns = ['lat', 'lng', 'district', 'address', 'cameraType', 'defaultAzimuth']
filtered_df = df[required_columns].dropna()

# Преобразование данных в список словарей
data = filtered_df.to_dict(orient='records')

# Сохранение данных в JSON-файл
output_file = 'v.json'
with open(output_file, 'w', encoding='utf-8') as f:
    pd.DataFrame(data).to_json(f, orient='records', force_ascii=False, indent=4)

print(f"JSON-файл {output_file} успешно создан.")